In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil

import keras
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

#from models.keras_ssd7 import build_model
#from models.keras_ssd7_copy import build_model  #Creating model that can be used for pruning

from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from data_generator.data_augmentation_chain_variable_input_size import DataAugmentationVariableInputSize
from data_generator.data_augmentation_chain_constant_input_size import DataAugmentationConstantInputSize
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation

import os.path
from os import path



%matplotlib inline

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False




### Pruning imports start

import tensorflow_model_optimization as tfmot  #Adding pruning imports

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude   #Adding pruning imports
    
from tensorflow import keras
    
### Pruning imports end

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [4]:
import keras.backend as K
K.clear_session()

from tensorflow.python import keras

model = keras.models.Sequential()
#add model layers
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu',input_shape=(28,28,1)))
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 203,434
Trainable params: 203,434
Non-trainable params: 0
_________________________________________________________________


In [6]:
    import tensorflow_model_optimization as tfmot  #Adding pruning imports

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude   #Adding pruning imports
    
    input_shape=[10]

    epochs = 2
    num_images = 18000
    end_step = np.ceil(num_images / 8).astype(np.int32) * epochs
    print("End_step -> ",end_step)

    # Define model for pruning.
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }
    
    
    model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu',input_shape=(28,28,1)), **pruning_params),
  
])
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(64, kernel_size=3, activation='relu',input_shape=(28,28,1)), **pruning_params)
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(64, kernel_size=3, activation='relu'), **pruning_params)

    
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), **pruning_params))
    #model_for_pruning = prune_low_magnitude(keras.layers.MaxPooling2D(pool_size=(2, 2)), **pruning_params)

    #model_for_pruning = keras.Sequential([prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params)])
    #model_for_pruning = keras.Sequential([prune_low_magnitude(conv1, **pruning_params)])

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    model_for_pruning.summary()

End_step ->  4500
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_2 (None, 26, 26, 64)        1218      
Total params: 1,218
Trainable params: 640
Non-trainable params: 578
_________________________________________________________________


In [70]:
    import tensorflow_model_optimization as tfmot  #Adding pruning imports

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude   #Adding pruning imports
    


    epochs = 2
    num_images = 18000
    end_step = np.ceil(num_images / 8).astype(np.int32) * epochs
    print("End_step -> ",end_step)

    # Define model for pruning.
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }

    model_for_pruning = prune_low_magnitude(model, **pruning_params)
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(64, kernel_size=3, activation='relu'), **pruning_params)

    
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), **pruning_params))
    #model_for_pruning = prune_low_magnitude(keras.layers.MaxPooling2D(pool_size=(2, 2)), **pruning_params)

    #model_for_pruning = keras.Sequential([prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params)])
    #model_for_pruning = keras.Sequential([prune_low_magnitude(conv1, **pruning_params)])

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    model_for_pruning.summary()

End_step ->  4500


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 480, 3)]     0         
_________________________________________________________________
prune_low_magnitude_identity (None, 300, 480, 3)       1         
_________________________________________________________________
prune_low_magnitude_conv1 (P (None, 300, 480, 32)      4834      
_________________________________________________________________
prune_low_magnitude_bn1 (Pru (None, 300, 480, 32)      129       
_________________________________________________________________
prune_low_magnitude_elu1 (Pr (None, 300, 480, 32)      1         
_________________________________________________________________
prune_low_magnitude_pool1 (P (None, 150, 240, 32)      1         
_________________________________________________________________
prune_low_magnitude_conv2 (P (None, 150, 240, 48)      27698 

In [2]:
    from __future__ import division
    import numpy as np
    #from keras.models import Model
    #from keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, ELU, Reshape, Concatenate, Activation
    from keras.regularizers import l2
    import keras.backend as K

    from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
    from keras_layers.keras_layer_DecodeDetections import DecodeDetections
    from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

    K.clear_session() # Clear previous models from memory.

    img_height = 300 # Height of the input images
    img_width = 480 # Width of the input images
    img_channels = 3 # Number of color channels of the input images
    intensity_mean = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
    intensity_range = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
    n_classes = 5 # Number of positive classes
    scales = [0.08, 0.16, 0.32, 0.64, 0.96] # An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
    aspect_ratios = [0.5, 1.0, 2.0] # The list of aspect ratios for the anchor boxes
    two_boxes_for_ar1 = True # Whether or not you want to generate two anchor boxes for aspect ratio 1
    steps = None # In case you'd like to set the step sizes for the anchor box grids manually; not recommended
    offsets = None # In case you'd like to set the offsets for the anchor box grids manually; not recommended
    clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
    variances = [1.0, 1.0, 1.0, 1.0] # The list of variances by which the encoded target coordinates are scaled
    normalize_coords = True # Whether or not the model is supposed to use coordinates relative to the image size


    l2_regularization=0.0
    image_size=(img_height, img_width, img_channels)
    aspect_ratios_global=[0.5, 1.0, 2.0]
    aspect_ratios_per_layer=None
    min_scale=0.1
    max_scale=0.9
    subtract_mean=None
    divide_by_stddev=None
    swap_channels=False
    confidence_thresh=0.01
    iou_threshold=0.45
    top_k=200
    nms_max_output_size=400
    return_predictor_sizes=False
    coords='centroids'
    mode='training'


    n_predictor_layers = 4 # The number of predictor conv layers in the network
    n_classes += 1 # Account for the background class.
    l2_reg = l2_regularization # Make the internal name shorter.
    img_height, img_width, img_channels = image_size[0], image_size[1], image_size[2]


    ############################################################################
    # Get a few exceptions out of the way.
    ############################################################################

    if aspect_ratios_global is None and aspect_ratios_per_layer is None:
        raise ValueError("`aspect_ratios_global` and `aspect_ratios_per_layer` cannot both be None. At least one needs to be specified.")
    if aspect_ratios_per_layer:
        if len(aspect_ratios_per_layer) != n_predictor_layers:
            raise ValueError("It must be either aspect_ratios_per_layer is None or len(aspect_ratios_per_layer) == {}, but len(aspect_ratios_per_layer) == {}.".format(n_predictor_layers, len(aspect_ratios_per_layer)))

    if (min_scale is None or max_scale is None) and scales is None:
        raise ValueError("Either `min_scale` and `max_scale` or `scales` need to be specified.")
    if scales:
        if len(scales) != n_predictor_layers+1:
            raise ValueError("It must be either scales is None or len(scales) == {}, but len(scales) == {}.".format(n_predictor_layers+1, len(scales)))
    else: # If no explicit list of scaling factors was passed, compute the list of scaling factors from `min_scale` and `max_scale`
        scales = np.linspace(min_scale, max_scale, n_predictor_layers+1)

    if len(variances) != 4: # We need one variance value for each of the four box coordinates
        raise ValueError("4 variance values must be pased, but {} values were received.".format(len(variances)))
    variances = np.array(variances)
    if np.any(variances <= 0):
        raise ValueError("All variances must be >0, but the variances given are {}".format(variances))

    if (not (steps is None)) and (len(steps) != n_predictor_layers):
        raise ValueError("You must provide at least one step value per predictor layer.")

    if (not (offsets is None)) and (len(offsets) != n_predictor_layers):
        raise ValueError("You must provide at least one offset value per predictor layer.")

    ############################################################################
    # Compute the anchor box parameters.
    ############################################################################

    # Set the aspect ratios for each predictor layer. These are only needed for the anchor box layers.
    if aspect_ratios_per_layer:
        aspect_ratios = aspect_ratios_per_layer
    else:
        aspect_ratios = [aspect_ratios_global] * n_predictor_layers

    # Compute the number of boxes to be predicted per cell for each predictor layer.
    # We need this so that we know how many channels the predictor layers need to have.
    if aspect_ratios_per_layer:
        n_boxes = []
        for ar in aspect_ratios_per_layer:
            if (1 in ar) & two_boxes_for_ar1:
                n_boxes.append(len(ar) + 1) # +1 for the second box for aspect ratio 1
            else:
                n_boxes.append(len(ar))
    else: # If only a global aspect ratio list was passed, then the number of boxes is the same for each predictor layer
        if (1 in aspect_ratios_global) & two_boxes_for_ar1:
            n_boxes = len(aspect_ratios_global) + 1
        else:
            n_boxes = len(aspect_ratios_global)
        n_boxes = [n_boxes] * n_predictor_layers

    if steps is None:
        steps = [None] * n_predictor_layers
    if offsets is None:
        offsets = [None] * n_predictor_layers

    ############################################################################
    # Define functions for the Lambda layers below.
    ############################################################################

    def identity_layer(tensor):
        return tensor

    def input_mean_normalization(tensor):
        return tensor - np.array(subtract_mean)

    def input_stddev_normalization(tensor):
        return tensor / np.array(divide_by_stddev)

    def input_channel_swap(tensor):
        if len(swap_channels) == 3:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]]], axis=-1)
        elif len(swap_channels) == 4:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]], tensor[...,swap_channels[3]]], axis=-1)

    ############################################################################
    # Build the network.
    ############################################################################

    x = keras.layers.Input(shape=(img_height, img_width, img_channels))

    # The following identity layer is only needed so that the subsequent lambda layers can be optional.
    x1 = keras.layers.Lambda(identity_layer, output_shape=(img_height, img_width, img_channels), name='identity_layer')(x)
    if not (subtract_mean is None):
        x1 = keras.layers.Lambda(input_mean_normalization, output_shape=(img_height, img_width, img_channels), name='input_mean_normalization')(x1)
    if not (divide_by_stddev is None):
        x1 = keras.layers.Lambda(input_stddev_normalization, output_shape=(img_height, img_width, img_channels), name='input_stddev_normalization')(x1)
    if swap_channels:
        x1 = keras.layers.Lambda(input_channel_swap, output_shape=(img_height, img_width, img_channels), name='input_channel_swap')(x1)

    conv1 = keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv1')(x1)
    conv1 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn1')(conv1) # Tensorflow uses filter format [filter_height, filter_width, in_channels, out_channels], hence axis = 3
    conv1 = keras.layers.ELU(name='elu1')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool1')(conv1)
    
    conv2 = keras.layers.Conv2D(48, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv2')(pool1)
    conv2 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn2')(conv2)
    conv2 = keras.layers.ELU(name='elu2')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool2')(conv2)

    ############################################################################
    # Pruning begin
    ############################################################################




    #model_for_pruning = prune_low_magnitude(model, **pruning_params)
    
    #model_for_pruning = tf.keras.Sequential([
  #tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params),
  
#])
    
    
    
    
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), **pruning_params))
    #model_for_pruning = prune_low_magnitude(keras.layers.MaxPooling2D(pool_size=(2, 2)), **pruning_params)

    #model_for_pruning = keras.Sequential([prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params)])
    #model_for_pruning = keras.Sequential([prune_low_magnitude(conv1, **pruning_params)])

    # `prune_low_magnitude` requires a recompile.


    
    
    ############################################################################
    # Pruning end
    ############################################################################


    if mode == 'training':
        model = keras.models.Model(inputs=x, outputs=pool2)
        model.summary()
        print(model.layers[2].name)
        
        





Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 480, 3)]     0         
_________________________________________________________________
identity_layer (Lambda)      (None, 300, 480, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 300, 480, 32)      2432      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 300, 480, 32)      128       
_________________________________________________________________
elu1 (ELU)                   (None, 300, 480, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 150, 240, 32)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 150, 240, 48)      138

In [24]:
    model_for_pruning = tf.keras.models.Model([
  tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params),
  
])
    
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    #model_for_pruning.summary()

In [37]:
model.summary()
# Use `prune_low_magnitude` to make the `MyDenseLayer` layer train with pruning.

model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(keras.layers.Conv2D(32, (5, 5)))])

model_for_pruning.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 480, 3)]     0         
_________________________________________________________________
identity_layer (Lambda)      (None, 300, 480, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 300, 480, 32)      2432      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 300, 480, 32)      128       
_________________________________________________________________
elu1 (ELU)                   (None, 300, 480, 32)      0         
Total params: 2,560
Trainable params: 2,496
Non-trainable params: 64
_________________________________________________________________


ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [3]:
# Helper function uses `prune_low_magnitude` to make only the 
# Dense layers train with pruning.
def apply_pruning_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    model,
    clone_function=apply_pruning_to_dense,
)

model_for_pruning.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 480, 3)]     0         
_________________________________________________________________
identity_layer (Lambda)      (None, 300, 480, 3)       0         
_________________________________________________________________
prune_low_magnitude_conv1 (P (None, 300, 480, 32)      4834      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 300, 480, 32)      128       
_________________________________________________________________
elu1 (ELU)                   (None, 300, 480, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 150, 240, 32)      0         
_________________________________________________________________
prune_low_magnitude_conv2 (P (None, 150, 240, 48)      2769

In [10]:
    import tensorflow_model_optimization as tfmot  #Adding pruning imports

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude   #Adding pruning imports
    
    from tensorflow import keras

    epochs = 2
    num_images = 18000
    end_step = np.ceil(num_images / 8).astype(np.int32) * epochs
    print("End_step -> ",end_step)

    # Define model for pruning.
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }

    #model_for_pruning = prune_low_magnitude(model, **pruning_params)
    
    model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params),
  
])
    
    
    
    
    #model_for_pruning = prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), **pruning_params))
    #model_for_pruning = prune_low_magnitude(keras.layers.MaxPooling2D(pool_size=(2, 2)), **pruning_params)

    #model_for_pruning = keras.Sequential([prune_low_magnitude(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg)), **pruning_params)])
    #model_for_pruning = keras.Sequential([prune_low_magnitude(conv1, **pruning_params)])

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    model_for_pruning.summary()

End_step ->  4500


ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [73]:
    from __future__ import division
    import numpy as np
    #from keras.models import Model
    #from keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, ELU, Reshape, Concatenate, Activation
    from keras.regularizers import l2
    import keras.backend as K
    
    from tensorflow import keras

    from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
    from keras_layers.keras_layer_DecodeDetections import DecodeDetections
    from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

    K.clear_session() # Clear previous models from memory.

    img_height = 300 # Height of the input images
    img_width = 480 # Width of the input images
    img_channels = 3 # Number of color channels of the input images
    intensity_mean = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
    intensity_range = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
    n_classes = 5 # Number of positive classes
    scales = [0.08, 0.16, 0.32, 0.64, 0.96] # An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
    aspect_ratios = [0.5, 1.0, 2.0] # The list of aspect ratios for the anchor boxes
    two_boxes_for_ar1 = True # Whether or not you want to generate two anchor boxes for aspect ratio 1
    steps = None # In case you'd like to set the step sizes for the anchor box grids manually; not recommended
    offsets = None # In case you'd like to set the offsets for the anchor box grids manually; not recommended
    clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
    variances = [1.0, 1.0, 1.0, 1.0] # The list of variances by which the encoded target coordinates are scaled
    normalize_coords = True # Whether or not the model is supposed to use coordinates relative to the image size


    l2_regularization=0.0
    image_size=(img_height, img_width, img_channels)
    aspect_ratios_global=[0.5, 1.0, 2.0]
    aspect_ratios_per_layer=None
    min_scale=0.1
    max_scale=0.9
    subtract_mean=None
    divide_by_stddev=None
    swap_channels=False
    confidence_thresh=0.01
    iou_threshold=0.45
    top_k=200
    nms_max_output_size=400
    return_predictor_sizes=False
    coords='centroids'
    mode='training'


    n_predictor_layers = 4 # The number of predictor conv layers in the network
    n_classes += 1 # Account for the background class.
    l2_reg = l2_regularization # Make the internal name shorter.
    img_height, img_width, img_channels = image_size[0], image_size[1], image_size[2]


    ############################################################################
    # Get a few exceptions out of the way.
    ############################################################################

    if aspect_ratios_global is None and aspect_ratios_per_layer is None:
        raise ValueError("`aspect_ratios_global` and `aspect_ratios_per_layer` cannot both be None. At least one needs to be specified.")
    if aspect_ratios_per_layer:
        if len(aspect_ratios_per_layer) != n_predictor_layers:
            raise ValueError("It must be either aspect_ratios_per_layer is None or len(aspect_ratios_per_layer) == {}, but len(aspect_ratios_per_layer) == {}.".format(n_predictor_layers, len(aspect_ratios_per_layer)))

    if (min_scale is None or max_scale is None) and scales is None:
        raise ValueError("Either `min_scale` and `max_scale` or `scales` need to be specified.")
    if scales:
        if len(scales) != n_predictor_layers+1:
            raise ValueError("It must be either scales is None or len(scales) == {}, but len(scales) == {}.".format(n_predictor_layers+1, len(scales)))
    else: # If no explicit list of scaling factors was passed, compute the list of scaling factors from `min_scale` and `max_scale`
        scales = np.linspace(min_scale, max_scale, n_predictor_layers+1)

    if len(variances) != 4: # We need one variance value for each of the four box coordinates
        raise ValueError("4 variance values must be pased, but {} values were received.".format(len(variances)))
    variances = np.array(variances)
    if np.any(variances <= 0):
        raise ValueError("All variances must be >0, but the variances given are {}".format(variances))

    if (not (steps is None)) and (len(steps) != n_predictor_layers):
        raise ValueError("You must provide at least one step value per predictor layer.")

    if (not (offsets is None)) and (len(offsets) != n_predictor_layers):
        raise ValueError("You must provide at least one offset value per predictor layer.")

    ############################################################################
    # Compute the anchor box parameters.
    ############################################################################

    # Set the aspect ratios for each predictor layer. These are only needed for the anchor box layers.
    if aspect_ratios_per_layer:
        aspect_ratios = aspect_ratios_per_layer
    else:
        aspect_ratios = [aspect_ratios_global] * n_predictor_layers

    # Compute the number of boxes to be predicted per cell for each predictor layer.
    # We need this so that we know how many channels the predictor layers need to have.
    if aspect_ratios_per_layer:
        n_boxes = []
        for ar in aspect_ratios_per_layer:
            if (1 in ar) & two_boxes_for_ar1:
                n_boxes.append(len(ar) + 1) # +1 for the second box for aspect ratio 1
            else:
                n_boxes.append(len(ar))
    else: # If only a global aspect ratio list was passed, then the number of boxes is the same for each predictor layer
        if (1 in aspect_ratios_global) & two_boxes_for_ar1:
            n_boxes = len(aspect_ratios_global) + 1
        else:
            n_boxes = len(aspect_ratios_global)
        n_boxes = [n_boxes] * n_predictor_layers

    if steps is None:
        steps = [None] * n_predictor_layers
    if offsets is None:
        offsets = [None] * n_predictor_layers

    ############################################################################
    # Define functions for the Lambda layers below.
    ############################################################################

    def identity_layer(tensor):
        return tensor

    def input_mean_normalization(tensor):
        return tensor - np.array(subtract_mean)

    def input_stddev_normalization(tensor):
        return tensor / np.array(divide_by_stddev)

    def input_channel_swap(tensor):
        if len(swap_channels) == 3:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]]], axis=-1)
        elif len(swap_channels) == 4:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]], tensor[...,swap_channels[3]]], axis=-1)

    ############################################################################
    # Build the network.
    ############################################################################

    x = keras.layers.Input(shape=(img_height, img_width, img_channels))

    # The following identity layer is only needed so that the subsequent lambda layers can be optional.
    x1 = keras.layers.Lambda(identity_layer, output_shape=(img_height, img_width, img_channels), name='identity_layer')(x)
    if not (subtract_mean is None):
        x1 = keras.layers.Lambda(input_mean_normalization, output_shape=(img_height, img_width, img_channels), name='input_mean_normalization')(x1)
    if not (divide_by_stddev is None):
        x1 = keras.layers.Lambda(input_stddev_normalization, output_shape=(img_height, img_width, img_channels), name='input_stddev_normalization')(x1)
    if swap_channels:
        x1 = keras.layers.Lambda(input_channel_swap, output_shape=(img_height, img_width, img_channels), name='input_channel_swap')(x1)

    conv1 = keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv1')(x1)
    conv1 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn1')(conv1) # Tensorflow uses filter format [filter_height, filter_width, in_channels, out_channels], hence axis = 3
    conv1 = keras.layers.ELU(name='elu1')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool1')(conv1)
    
    ############################################################################
    # Pruning begin
    ############################################################################
    
    
    
    
    ############################################################################
    # Pruning end
    ############################################################################

    conv2 = keras.layers.Conv2D(48, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv2')(pool1)
    conv2 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn2')(conv2)
    conv2 = keras.layers.ELU(name='elu2')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool2')(conv2)

    conv3 = keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv3')(pool2)
    conv3 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn3')(conv3)
    conv3 = keras.layers.ELU(name='elu3')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool3')(conv3)

    conv4 = keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv4')(pool3)
    conv4 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn4')(conv4)
    conv4 = keras.layers.ELU(name='elu4')(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool4')(conv4)

    conv5 = keras.layers.Conv2D(48, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv5')(pool4)
    conv5 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn5')(conv5)
    conv5 = keras.layers.ELU(name='elu5')(conv5)
    pool5 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool5')(conv5)

    conv6 = keras.layers.Conv2D(48, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv6')(pool5)
    conv6 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn6')(conv6)
    conv6 = keras.layers.ELU(name='elu6')(conv6)
    pool6 = keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool6')(conv6)

    conv7 = keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv7')(pool6)
    conv7 = keras.layers.BatchNormalization(axis=3, momentum=0.99, name='bn7')(conv7)
    conv7 = keras.layers.ELU(name='elu7')(conv7)

    # The next part is to add the convolutional predictor layers on top of the base network
    # that we defined above. Note that I use the term "base network" differently than the paper does.
    # To me, the base network is everything that is not convolutional predictor layers or anchor
    # box layers. In this case we'll have four predictor layers, but of course you could
    # easily rewrite this into an arbitrarily deep base network and add an arbitrary number of
    # predictor layers on top of the base network by simply following the pattern shown here.

    # Build the convolutional predictor layers on top of conv layers 4, 5, 6, and 7.
    # We build two predictor layers on top of each of these layers: One for class prediction (classification), one for box coordinate prediction (localization)
    # We precidt `n_classes` confidence values for each box, hence the `classes` predictors have depth `n_boxes * n_classes`
    # We predict 4 box coordinates for each box, hence the `boxes` predictors have depth `n_boxes * 4`
    # Output shape of `classes`: `(batch, height, width, n_boxes * n_classes)`
    classes4 = keras.layers.Conv2D(n_boxes[0] * n_classes, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='classes4')(conv4)
    classes5 = keras.layers.Conv2D(n_boxes[1] * n_classes, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='classes5')(conv5)
    classes6 = keras.layers.Conv2D(n_boxes[2] * n_classes, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='classes6')(conv6)
    classes7 = keras.layers.Conv2D(n_boxes[3] * n_classes, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='classes7')(conv7)
    # Output shape of `boxes`: `(batch, height, width, n_boxes * 4)`
    boxes4 = keras.layers.Conv2D(n_boxes[0] * 4, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='boxes4')(conv4)
    boxes5 = keras.layers.Conv2D(n_boxes[1] * 4, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='boxes5')(conv5)
    boxes6 = keras.layers.Conv2D(n_boxes[2] * 4, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='boxes6')(conv6)
    boxes7 = keras.layers.Conv2D(n_boxes[3] * 4, (3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='boxes7')(conv7)

    # Generate the anchor boxes
    # Output shape of `anchors`: `(batch, height, width, n_boxes, 8)`
    anchors4 = AnchorBoxes(img_height, img_width, this_scale=scales[0], next_scale=scales[1], aspect_ratios=aspect_ratios[0],
                           two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[0], this_offsets=offsets[0],
                           clip_boxes=clip_boxes, variances=variances, coords=coords, normalize_coords=normalize_coords, name='anchors4')(boxes4)
    anchors5 = AnchorBoxes(img_height, img_width, this_scale=scales[1], next_scale=scales[2], aspect_ratios=aspect_ratios[1],
                           two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[1], this_offsets=offsets[1],
                           clip_boxes=clip_boxes, variances=variances, coords=coords, normalize_coords=normalize_coords, name='anchors5')(boxes5)
    anchors6 = AnchorBoxes(img_height, img_width, this_scale=scales[2], next_scale=scales[3], aspect_ratios=aspect_ratios[2],
                           two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[2], this_offsets=offsets[2],
                           clip_boxes=clip_boxes, variances=variances, coords=coords, normalize_coords=normalize_coords, name='anchors6')(boxes6)
    anchors7 = AnchorBoxes(img_height, img_width, this_scale=scales[3], next_scale=scales[4], aspect_ratios=aspect_ratios[3],
                           two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[3], this_offsets=offsets[3],
                           clip_boxes=clip_boxes, variances=variances, coords=coords, normalize_coords=normalize_coords, name='anchors7')(boxes7)

    # Reshape the class predictions, yielding 3D tensors of shape `(batch, height * width * n_boxes, n_classes)`
    # We want the classes isolated in the last axis to perform softmax on them
    classes4_reshaped = keras.layers.Reshape((-1, n_classes), name='classes4_reshape')(classes4)
    classes5_reshaped = keras.layers.Reshape((-1, n_classes), name='classes5_reshape')(classes5)
    classes6_reshaped = keras.layers.Reshape((-1, n_classes), name='classes6_reshape')(classes6)
    classes7_reshaped = keras.layers.Reshape((-1, n_classes), name='classes7_reshape')(classes7)
    # Reshape the box coordinate predictions, yielding 3D tensors of shape `(batch, height * width * n_boxes, 4)`
    # We want the four box coordinates isolated in the last axis to compute the smooth L1 loss
    boxes4_reshaped = keras.layers.Reshape((-1, 4), name='boxes4_reshape')(boxes4)
    boxes5_reshaped = keras.layers.Reshape((-1, 4), name='boxes5_reshape')(boxes5)
    boxes6_reshaped = keras.layers.Reshape((-1, 4), name='boxes6_reshape')(boxes6)
    boxes7_reshaped = keras.layers.Reshape((-1, 4), name='boxes7_reshape')(boxes7)
    # Reshape the anchor box tensors, yielding 3D tensors of shape `(batch, height * width * n_boxes, 8)`
    anchors4_reshaped = keras.layers.Reshape((-1, 8), name='anchors4_reshape')(anchors4)
    anchors5_reshaped = keras.layers.Reshape((-1, 8), name='anchors5_reshape')(anchors5)
    anchors6_reshaped = keras.layers.Reshape((-1, 8), name='anchors6_reshape')(anchors6)
    anchors7_reshaped = keras.layers.Reshape((-1, 8), name='anchors7_reshape')(anchors7)

    # Concatenate the predictions from the different layers and the assosciated anchor box tensors
    # Axis 0 (batch) and axis 2 (n_classes or 4, respectively) are identical for all layer predictions,
    # so we want to concatenate along axis 1
    # Output shape of `classes_concat`: (batch, n_boxes_total, n_classes)
    classes_concat = keras.layers.Concatenate(axis=1, name='classes_concat')([classes4_reshaped,
                                                                 classes5_reshaped,
                                                                 classes6_reshaped,
                                                                 classes7_reshaped])

    # Output shape of `boxes_concat`: (batch, n_boxes_total, 4)
    boxes_concat = keras.layers.Concatenate(axis=1, name='boxes_concat')([boxes4_reshaped,
                                                             boxes5_reshaped,
                                                             boxes6_reshaped,
                                                             boxes7_reshaped])

    # Output shape of `anchors_concat`: (batch, n_boxes_total, 8)
    anchors_concat = keras.layers.Concatenate(axis=1, name='anchors_concat')([anchors4_reshaped,
                                                                 anchors5_reshaped,
                                                                 anchors6_reshaped,
                                                                 anchors7_reshaped])

    # The box coordinate predictions will go into the loss function just the way they are,
    # but for the class predictions, we'll apply a softmax activation layer first
    classes_softmax = keras.layers.Activation('softmax', name='classes_softmax')(classes_concat)

    # Concatenate the class and box coordinate predictions and the anchors to one large predictions tensor
    # Output shape of `predictions`: (batch, n_boxes_total, n_classes + 4 + 8)
    predictions = keras.layers.Concatenate(axis=2, name='predictions')([classes_softmax, boxes_concat, anchors_concat])
    
    #print("Value of x ->", x)
    #print("Output  ->", predictions)

    if mode == 'training':
        model = keras.models.Model(inputs=x, outputs=predictions)
        model.summary()
    elif mode == 'inference':
        decoded_predictions = DecodeDetections(confidence_thresh=confidence_thresh,
                                               iou_threshold=iou_threshold,
                                               top_k=top_k,
                                               nms_max_output_size=nms_max_output_size,
                                               coords=coords,
                                               normalize_coords=normalize_coords,
                                               img_height=img_height,
                                               img_width=img_width,
                                               name='decoded_predictions')(predictions)
        model = keras.models.Model(inputs=x, outputs=decoded_predictions)
    elif mode == 'inference_fast':
        decoded_predictions = DecodeDetectionsFast(confidence_thresh=confidence_thresh,
                                                   iou_threshold=iou_threshold,
                                                   top_k=top_k,
                                                   nms_max_output_size=nms_max_output_size,
                                                   coords=coords,
                                                   normalize_coords=normalize_coords,
                                                   img_height=img_height,
                                                   img_width=img_width,
                                                   name='decoded_predictions')(predictions)
        model = keras.models.Model(inputs=x, outputs=decoded_predictions)
        model.summary()
    else:
        raise ValueError("`mode` must be one of 'training', 'inference' or 'inference_fast', but received '{}'.".format(mode))

    if return_predictor_sizes:
        # The spatial dimensions are the same for the `classes` and `boxes` predictor layers.
        predictor_sizes = np.array([classes4._keras_shape[1:3],
                                    classes5._keras_shape[1:3],
                                    classes6._keras_shape[1:3],
                                    classes7._keras_shape[1:3]])
        model, predictor_sizes
    else:
        model
        


AttributeError: 'Node' object has no attribute 'output_masks'